In [1]:
%%html
<style>
  .array th { border: thick solid; font-family: monospace }
</style>

# Divide-and-Conquer and Sorting

In this lecture, we'll explore in more detail the recursive divide-and-conquer paradigm that we introduced in Lecture 3.  Specifically, we'll look at the following sorting or sorting-related algorithms:

1. Merge Sort
2. In-place Merging
3. Nuts and Bolts Puzzle
4. Quicksort

Recall our three-step recipe for writing a recursive function:

 1. **Divide** the input space into at least two regions.
 2. Identify some regions as **base cases** and solve them immediately* (without recursion).
 3. Solve the remaining regions as **recursive cases**, where each makes one or more calls to the same function *but where the arguments have "gotten smaller"*.

## Merge Sort

We covered merge sort briefly at the end of Lecture 3, but let's look more closely at how it works.  Suppose we have an array of four elements that we want to sort into ascending order:

The idea with merge sort is to divide the array into two equal-sized subarrays:

<table class="array"><tr><th>1<th>4</tr></table>

<table class="array"><tr><th>3<th>7</tr></table>

Now we can sort the two subarrays recursively.  A list of size 2 is small enough that it could be covered by a base case: compare the two elements, and swap them if they're out of order.  In our example, the first subarray is already in order, while the second subarray is out of order so would get swapped.  Thus, after the two recursive calls, we'd have the following two arrays:

<table class="array"><tr><th>1<th>4</tr></table>

<table class="array"><tr><th>3<th>7</tr></table>

Now we come back to the first (topmost) call to merge sort, and our task is to merge these two *sorted* subarrays into one sorted array.  We do this using a merge algorithm, which simply compares the first elements of the two subarrays, repeatedly.  In our example, merge first compares `1` and `3`.  Because `1` is smaller, it outputs that to the merged list, and conceptually removes it from the first list.  What remains:

<table class="array"><tr><th>4</tr></table>

<table class="array"><tr><th>3<th>7</tr></table>

Merge again compares the first elements of the two subarrays, now `4` and `3`. This time, `3` from the second subarray is smaller, so it gets output and removed from its subarray.  What remains:

<table class="array"><tr><th>4</tr></table>

<table class="array"><tr><th>7</tr></table>

One more comparison, and `4` gets output and removed.  Now the first subarray is empty, so we output all the remaining elements from the second subarray, which here is just `7`.  If you've been following along, the final output order in our example is sorted in ascending order:

<table class="array"><tr><th>1<th>3<th>4<th>7</tr></table>

Viewed according to the general 3-step recipe, merge sort works as follows:

 1. *Divide the input space*: lists of length below two and at least two
 2. *Base cases*: length below two (the list is already sorted)
 3. *Recursive cases*: divide the input list roughly in half, so that each "half" is shorter than the original and thus legal to sort recursively

One note about the general recursion: we might happen upon an odd-length array, in which case we can't split it into two subarrays of exactly equal size.  In this case, we'll have to have subarrays that differ in size by 1.  We just need to make sure we can merge subarrays of unequal size.

### Recursion

Now let's look at the code to implement this algorithm, assuming a function to `merge` two sorted arrays:

In [2]:
def merge_sort(L, indent=1):
    """Returns a new sorted list containing the same elements as L."""
    print('.'*(indent-1), 'SORTING', L)

    # Base case: less than 2 elements, so already sorted
    if len(L) < 2:
        return L[:] # Makes a fresh copy of L, just in case the caller mutates L later.

    # Split L into two lists of approximately equal lengths.  (They might be off by one.)
    middle = len(L)//2
    left = L[:middle]
    right = L[middle:]
    print('.'*indent, 'SPLIT into', left, 'and', right)
        
    # Sort the two sides.
    left = merge_sort(left, indent+1)
    right = merge_sort(right, indent+1)
    print('.'*indent, 'SORTED into', left, 'and', right)
        
    # Merge the results.
    merged = merge(left, right)
    print('.'*indent, 'MERGED into', merged)
    return merged

### Merge

In [3]:
def merge(left, right):
    """Assumes left and right are sorted lists.  Returns a single new
    list built, in order, from the elements of left and right."""

    result = []
    left_i = right_i = 0  # indices into left and right, respectively
    
    # Loop while there are elements in both lists.
    while left_i < len(left) and right_i < len(right):
        # Copy smallest element to result.
        if left[left_i] < right[right_i]:
            result.append(left[left_i])
            left_i += 1
        else:
            result.append(right[right_i])
            right_i += 1

    # Copy over any remaining elements.
    # Only one of the lists has any elements remaining!
    result.extend(left[left_i:]) # Recall that extend adds all elements of the argument list.
    result.extend(right[right_i:])
    return result

In [4]:
assert merge([1, 2], [3, 4]) == [1, 2, 3, 4]
assert merge([1, 3], [2, 4]) == [1, 2, 3, 4]

In [5]:
assert merge_sort([1, 4, 7, 3]) == [1, 3, 4, 7]

 SORTING [1, 4, 7, 3]
. SPLIT into [1, 4] and [7, 3]
. SORTING [1, 4]
.. SPLIT into [1] and [4]
.. SORTING [1]
.. SORTING [4]
.. SORTED into [1] and [4]
.. MERGED into [1, 4]
. SORTING [7, 3]
.. SPLIT into [7] and [3]
.. SORTING [7]
.. SORTING [3]
.. SORTED into [7] and [3]
.. MERGED into [3, 7]
. SORTED into [1, 4] and [3, 7]
. MERGED into [1, 3, 4, 7]


In [6]:
assert merge_sort([23, 3, 45, 7, 6, 11, 14, 12]) == [3, 6, 7, 11, 12, 14, 23, 45]

 SORTING [23, 3, 45, 7, 6, 11, 14, 12]
. SPLIT into [23, 3, 45, 7] and [6, 11, 14, 12]
. SORTING [23, 3, 45, 7]
.. SPLIT into [23, 3] and [45, 7]
.. SORTING [23, 3]
... SPLIT into [23] and [3]
... SORTING [23]
... SORTING [3]
... SORTED into [23] and [3]
... MERGED into [3, 23]
.. SORTING [45, 7]
... SPLIT into [45] and [7]
... SORTING [45]
... SORTING [7]
... SORTED into [45] and [7]
... MERGED into [7, 45]
.. SORTED into [3, 23] and [7, 45]
.. MERGED into [3, 7, 23, 45]
. SORTING [6, 11, 14, 12]
.. SPLIT into [6, 11] and [14, 12]
.. SORTING [6, 11]
... SPLIT into [6] and [11]
... SORTING [6]
... SORTING [11]
... SORTED into [6] and [11]
... MERGED into [6, 11]
.. SORTING [14, 12]
... SPLIT into [14] and [12]
... SORTING [14]
... SORTING [12]
... SORTED into [14] and [12]
... MERGED into [12, 14]
.. SORTED into [6, 11] and [12, 14]
.. MERGED into [6, 11, 12, 14]
. SORTED into [3, 7, 23, 45] and [6, 11, 12, 14]
. MERGED into [3, 6, 7, 11, 12, 14, 23, 45]


## In-place Merging

One unsatisfying feature of `merge_sort` is that it outputs a new list separate from the input list.  In fact, it makes many different lists along the way, which can waste both time and memory.  Compare this with the `list.sort()` interface, which sorts a list **in-place**, i.e., by modifying the existing list instead of creating a new one and without using lots of auxiliary space.  Could we modify merge sort to work in place?

It's easy enough to avoid making the `left` and `right` lists in `merge_sort`, by keeping track of left and right indices instead:

In [7]:
def inplace_merge_sort(L, left=0, right=None, indent=1):
    """Sorts L[left:right] in place, defaulting to all of L."""
    if right is None: right = len(L)
    print('.'*(indent-1), 'SORTING', L[left:right])

    # Base case: less than 2 elements, so already sorted
    if right - left < 2: return
    
    # Split L into two lists of approximately equal lengths.  (They might be off by one.)
    middle = (left + right) // 2
    print('.'*indent, 'SPLIT into', L[left:middle], 'and', L[middle:right])
        
    # Sort the two sides.
    inplace_merge_sort(L, left, middle, indent+1)
    inplace_merge_sort(L, middle, right, indent+1)
    print('.'*indent, 'SORTED into', L[left:middle], 'and', L[middle:right])
        
    # Merge the results.
    inplace_merge(L, left, middle, right, '.'*indent)
    print('.'*indent, 'MERGED into', L[left:right])

The challenge is to perform the merge in-place.  Just to test the code we have so far, here's a cheating function that merges "in place", but by creating an intermediate array to store the result and then putting it back in the list (so not really "in place").

In [8]:
def inplace_merge(L, left, middle, right, prefix = ''):
    """Cheating function to merge sorted L[left:middle] and
    sorted L[middle:right] into sorted L[left:right]."""
    L[left:right] = merge(L[left:middle], L[middle:right])

In [9]:
L = [1, 4, 7, 3]
inplace_merge_sort(L)
assert L == [1, 3, 4, 7]

 SORTING [1, 4, 7, 3]
. SPLIT into [1, 4] and [7, 3]
. SORTING [1, 4]
.. SPLIT into [1] and [4]
.. SORTING [1]
.. SORTING [4]
.. SORTED into [1] and [4]
.. MERGED into [1, 4]
. SORTING [7, 3]
.. SPLIT into [7] and [3]
.. SORTING [7]
.. SORTING [3]
.. SORTED into [7] and [3]
.. MERGED into [3, 7]
. SORTED into [1, 4] and [3, 7]
. MERGED into [1, 3, 4, 7]


### Recursive Merging

The big idea is that in-place merging can itself by solved by a divide-and-conquer approach.  We're given an array `L[left:right]` that consists of two sorted subarrays `L[left:middle]` and `L[middle:right]` concatenated together.  Here's an example where they're the same size, though they may not be:

<table class="array">
    <tr><td colspan="6"><td colspan="6" style="border-left: solid 6px purple"></tr>
    <tr><th style="background: #ff0">10<th style="background: #ff0">15<th style="background: #ff0">19<th style="background: #ff0">23<th style="background: #ff0">26<th style="background: #ff0">32
        <th style="border-left: solid 6px purple; background: #92d050">4<th style="background: #93d050">9<th style="background: #93d050">16<th style="background: #93d050">30<th style="background: #93d050">39<th style="background: #93d050">47
</tr>
    <tr><td colspan="6"><td colspan="6" style="border-left: solid 6px purple"></tr>
</table>

Our goal is to re-arrange the elements so that we can recursively perform a merge entirely within the left subarray and another merge within the right subarray, and have a fully sorted/merged array.  For this to work, we'd need all the elements in the left subarray to be smaller than all the elements in the right subarray.  It turns out that this is always possible with one "big swap".  Let's see why.

We might be lucky, and all the elements in the left subarray are already smaller than all the elements in the right subarray.  Because the left and right subarrays are sorted, we can tell whether we're lucky by a single comparison: check whether the rightmost element of the left subarray is less than the leftmost element of the right subarray.  If it is, we're lucky, and we're actually done (no recursion necessary: the array is already sorted).

If not, we know that these two elements (the rightmost element of the left subarray and the leftmost element of the right subarray) need to swap which arrays they're in: the left element is too big and the right element is too small.  Now let's forget about those elements and continue: check whether the remaining elements in the left subarray are all less than the remaining elements in the right subarray, by comparing the extreme elements.  If yes, we can just swap the two elements previously identified.

If not, we've found two more elements that need to swap subarrays.  Let's just keep going until we find the desired inequality (remaining left subarray elements are all smaller than remaining right subarray elements), and then swap all the identified out-of-order elements while preserving their relative order (so that they remain a sorted subarray).  In our example, we happen to make it exactly half way:

<img style="width: 50%" src="img/inplace_merge_6.png">

By this big swap, we've achieved the desired property: all the elements in the left subarray are now smaller than all the elements in the right subarray.  The subarrays aren't sorted yet, but they *do* decompose into two sorted subsubarrays.  (In this example, they all happen to be the same size, but that won't necessary be the case.)

How do we sort the subarrays?  Recursively!  This is exactly the setup for calling in-place merge recursively: each subarray consists of two known subsubarrays, each of which is sorted.  Because we've arranged for the left subarray to have smaller elements than the right subarray, once we recursively sort/merge each of them, we'll have a completely sorted array.

Here's the next level of recursion in our example.  (Now we finally get arrays of different sizes.)

<img style="width: 50%" src="img/inplace_merge_3_3.png">

The right subarray is now sorted: in the next recursion, we'll detect that each subsubarray is actually already ordered correctly (the left subsubsubarray's elements are smaller than the right subsubsubarray's elements), and the recursion will stop.  The left subarray isn't quite sorted yet, but after one more level of recursion and a big swap in each, we'll have a sorted subarray.  Awesome!

### Recursion

Now let's look at the code to implement this algorithm, assuming a function to `big_swap`:

In [24]:
def inplace_merge(L, left, middle, right, prefix = ''):
    """Merge sorted L[left:middle] and sorted L[middle:right]
    into sorted L[left:right]."""
    prefix += ':'
    print(f'{prefix} MERGE {L[left:middle]} AND {L[middle:right]} WITHIN {L}')
    # Base case: no elements in left part L[left:middle]
    if middle - left < 1: return
    # Base case: no elements in right part L[middle:right]
    if right - middle < 1: return
    # Recursive case.
    # Repeatedly move index left_i left in left part and
    # correspondingly move index right_i right in right part,
    # until we find L[left_i] <= L[right_i],
    # or we fall off left or right edge.
    left_i = middle - 1
    right_i = middle
    while left_i >= left and right_i < right and L[left_i] > L[right_i]:
        left_i -= 1
        right_i += 1
    # Move indices back one so that L[left_i] > L[right_i]
    # and this is the last such index.
    left_i += 1
    right_i -= 1
    # If left index hits left edge, check for more small elements on right.
    if left_i == left:
        while right_i < right and L[left_i] > L[right_i]:
            right_i += 1
        right_i -= 1  # fix overshoot
    # If right index hits right edge, check for more big elements on left.
    if right_i == right - 1:
        while left_i >= left and L[left_i] > L[right_i]:
            left_i -= 1
        left_i += 1  # fix overshoot
    # Now swap L[left_i:middle] with L[middle:right_i+1]
    big_swap(L, left_i, middle, right_i + 1, prefix)
    # Finally we can recurse.
    inplace_merge(L, left, left_i, middle, prefix)
    inplace_merge(L, middle, right_i + 1, right, prefix)

We can test this out with a fake (not in-place) implementation of `big_swap`:

In [27]:
def big_swap(L, left, middle, right, prefix = ''):
    """Cheating function to swap L[left:middle] with L[middle:right]"""
    L[left:right] = L[middle:right] + L[left:middle]

In [28]:
L = [10, 15, 19, 23, 26, 32, 4, 9, 16, 30, 39, 47]
inplace_merge(L, 0, 6, 12)
assert L == [4, 9, 10, 15, 16, 19, 23, 26, 30, 32, 39, 47]

: MERGE [10, 15, 19, 23, 26, 32] AND [4, 9, 16, 30, 39, 47] WITHIN [10, 15, 19, 23, 26, 32, 4, 9, 16, 30, 39, 47]
:: MERGE [10, 15, 19] AND [4, 9, 16] WITHIN [10, 15, 19, 4, 9, 16, 23, 26, 32, 30, 39, 47]
::: MERGE [10] AND [4, 9] WITHIN [10, 4, 9, 15, 19, 16, 23, 26, 32, 30, 39, 47]
:::: MERGE [] AND [4] WITHIN [4, 9, 10, 15, 19, 16, 23, 26, 32, 30, 39, 47]
:::: MERGE [9, 10] AND [] WITHIN [4, 9, 10, 15, 19, 16, 23, 26, 32, 30, 39, 47]
::: MERGE [15, 19] AND [16] WITHIN [4, 9, 10, 15, 19, 16, 23, 26, 32, 30, 39, 47]
:::: MERGE [15] AND [16] WITHIN [4, 9, 10, 15, 16, 19, 23, 26, 32, 30, 39, 47]
::::: MERGE [15] AND [] WITHIN [4, 9, 10, 15, 16, 19, 23, 26, 32, 30, 39, 47]
::::: MERGE [] AND [16] WITHIN [4, 9, 10, 15, 16, 19, 23, 26, 32, 30, 39, 47]
:::: MERGE [19] AND [] WITHIN [4, 9, 10, 15, 16, 19, 23, 26, 32, 30, 39, 47]
:: MERGE [23, 26, 32] AND [30, 39, 47] WITHIN [4, 9, 10, 15, 16, 19, 23, 26, 32, 30, 39, 47]
::: MERGE [23, 26] AND [30] WITHIN [4, 9, 10, 15, 16, 19, 23, 26, 30, 32

### Big Swap

A more subtle part of the code is to figure out how to do a big swap in terms of a bunch of little swaps.

#### Same size

Usually, including all the examples above, the two halves that need to be swapped have the same size, because we always advanced `left_i` and `right_i` together.  In this case, it's really easy: just swap each element in the left part with the corresponding element in the right part.  For example:

<table class="array">
    <!--<tr><td colspan="3"><td colspan="6" style="border-left: solid 6px purple"><td colspan="3" style="border-left: solid 6px purple"></tr>-->
    <tr><td colspan="3"><td colspan="3" style="border-left: solid 6px purple"><td colspan="3" style="border-left: solid 6px red"><td colspan="3" style="border-left: solid 6px purple"></tr>
    <tr><th style="background: #ff0">10<th style="background: #ff0">15<th style="background: #ff0">19
        <th style="border-left: solid 6px purple; background: #bdd7e3">23<th style="background: #bdd7e3">26<th style="background: #bdd7e3">32
        <th style="border-left: dashed 6px red; background: #f4b183">4<th style="background: #f4b183">9<th style="background: #f4b183">16
        <th style="border-left: solid 6px purple; background: #93d050">30<th style="background: #93d050">39<th style="background: #93d050">47
</tr>
    <tr><td colspan="3"><td colspan="3" style="border-left: solid 6px purple"><td colspan="3" style="border-left: solid 6px red"><td colspan="3" style="border-left: solid 6px purple"></tr>
    <!--<tr><td colspan="3"><td colspan="6" style="border-left: solid 6px purple"><td colspan="3" style="border-left: solid 6px purple"></tr>-->
</table>

We can swap `23` with `4`, `26` with `9`, and `32` with `16` to achieve the desired big swap:

<table class="array">
    <!--<tr><td colspan="3"><td colspan="6" style="border-left: solid 6px purple"><td colspan="3" style="border-left: solid 6px purple"></tr>-->
    <tr><td colspan="3"><td colspan="3" style="border-left: solid 6px purple"><td colspan="3" style="border-left: solid 6px red"><td colspan="3" style="border-left: solid 6px purple"></tr>
    <tr><th style="background: #ff0">10<th style="background: #ff0">15<th style="background: #ff0">19
        <th style="border-left: solid 6px purple; background: #f4b183">4<th style="background: #f4b183">9<th style="background: #f4b183">16
        <th style="border-left: dashed 6px red; background: #bdd7e3">23<th style="background: #bdd7e3">26<th style="background: #bdd7e3">32
        <th style="border-left: solid 6px purple; background: #93d050">30<th style="background: #93d050">39<th style="background: #93d050">47
</tr>
    <tr><td colspan="3"><td colspan="3" style="border-left: solid 6px purple"><td colspan="3" style="border-left: solid 6px red"><td colspan="3" style="border-left: solid 6px purple"></tr>
    <!--<tr><td colspan="3"><td colspan="6" style="border-left: solid 6px purple"><td colspan="3" style="border-left: solid 6px purple"></tr>-->
</table>

Unfortunately, we also need to handle the case where the two swapped subarrays are not the same size, because of the case where we run out of elements in one subarray, and more elements in the other subarray may need to swap (the "hitting left/right edge" cases in the code above).  This case is messier.

#### Size 1

Another easy case is when one subarray has size 1.  Then we can swap that one item repeatedly into the other subarray, until it reaches the end:

<table class="array">
    <tr><td colspan="1" style="border-left: solid 6px purple"><td colspan="3" style="border-left: solid 6px red; border-right: solid 6px purple"></tr>
    <tr><th style="border-left: solid 6px purple; background: #f4b183">4
        <th style="border-left: dashed 6px red; background: #bdd7e3">23<th style="background: #bdd7e3">26<th style="border-right: solid 6px purple; background: #bdd7e3">32
</tr>
    <tr><td colspan="1" style="border-left: solid 6px purple"><td colspan="3" style="border-left: solid 6px red; border-right: solid 6px purple"></tr>
</table>

<table class="array">
    <tr><th style="background: #bdd7e3">23
        <th style="background: #f4b183">4<th style="background: #bdd7e3">26<th style="background: #bdd7e3">32
</tr>
</table>

<table class="array">
    <tr><th style="background: #bdd7e3">23<th style="background: #bdd7e3">26
        <th style="background: #f4b183">4<th style="background: #bdd7e3">32
</tr>
</table>

<table class="array">
    <tr><td colspan="3" style="border-left: solid 6px purple"><td colspan="1" style="border-left: solid 6px red; border-right: solid 6px purple"></tr>
    <tr><th style="border-left: solid 6px purple; background: #bdd7e3">23<th style="background: #bdd7e3">26<th style="background: #bdd7e3">32
        <th style="border-left: dashed 6px red; border-right: solid 6px purple; background: #f4b183">4
</tr>
    <tr><td colspan="3" style="border-left: solid 6px purple"><td colspan="1" style="border-left: solid 6px red; border-right: solid 6px purple"></tr>
</table>

#### General case

We can generalize these two easy cases into one full solution.  First, among the two subarrays, take the one of smaller size.  Then swap that subarray with just the adjacent same-size chunk of the other subarray.  This pushes the smaller-size subarray in the correct direction, but won't get it all the way to the end.  But we can do the rest simply by recursing.  (This is tail recursion, so it could also be expressed efficiently with a `while` loop.)  For example:

<table class="array">
    <tr><td colspan="2" style="border-left: solid 6px purple"><td colspan="3" style="border-left: solid 6px red; border-right: solid 6px purple"></tr>
    <tr><th style="border-left: solid 6px purple; background: #f4b183">4<th style="background: #f4b183">9
        <th style="border-left: dashed 6px red; background: #bdd7e3">23<th style="background: #bdd7e3">26<th style="border-right: solid 6px purple; background: #bdd7e3">32
</tr>
    <tr><td colspan="2" style="border-left: solid 6px purple"><td colspan="3" style="border-left: solid 6px red; border-right: solid 6px purple"></tr>
</table>

The smaller-size subarray is the left subarray, so we swap it with the beginning of the right subarray by swapping `4` with `23` and `9` with `26`:

<table class="array">
    <tr><td colspan="2"><td colspan="2" style="border-left: solid 6px purple"><td colspan="1" style="border-left: solid 6px red; border-right: solid 6px purple"></tr>
    <tr><th style="background: #bdd7e3">23<th style="background: #bdd7e3">26
        <th style="border-left: solid 6px purple; background: #f4b183">4<th style="background: #f4b183">9
        <th style="border-left: dashed 6px red; border-right: solid 6px purple; background: #bdd7e3">32
</tr>
    <tr><td colspan="2"><td colspan="2" style="border-left: solid 6px purple"><td colspan="1" style="border-left: solid 6px red; border-right: solid 6px purple"></tr>
</table>

It remains to swap `[4, 9]` with `[32]`.  Now the right subarray (`32`) is the smaller one, so we swap it into the beginning of the left subarray (`9`):

<table class="array">
    <tr><td colspan="2"><td colspan="1" style="border-left: solid 6px purple"><td colspan="1" style="border-left: solid 6px red"><td colspan="1" style="border-left: solid 6px purple"></tr>
    <tr><th style="background: #bdd7e3">23<th style="background: #bdd7e3">26
        <th style="border-left: solid 6px purple; background: #f4b183">4
        <th style="border-left: dashed 6px red; background: #bdd7e3">32
        <th style="border-left: solid 6px purple; background: #f4b183">9
</tr>
    <tr><td colspan="2"><td colspan="1" style="border-left: solid 6px purple"><td colspan="1" style="border-left: solid 6px red"><td colspan="1" style="border-left: solid 6px purple"></tr>
</table>

It remains to swap `[4]` with `[32]`.  Now we have equal-size subarrays (both length 1), so we can swap corresponding elements (here just `4` and `32`):

<table class="array">
    <tr><td colspan="3" style="border-left: solid 6px purple"><td colspan="2" style="border-left: solid 6px red; border-right: solid 6px purple"></tr>
    <tr><th style="border-left: solid 6px purple; background: #bdd7e3">23<th style="background: #bdd7e3">26
        <th style="background: #bdd7e3">32
        <th style="border-left: dashed 6px red; background: #f4b183">4
        <th style="border-right: solid 6px purple; background: #f4b183">9
</tr>
    <tr><td colspan="3" style="border-left: solid 6px purple"><td colspan="2" style="border-left: solid 6px red; border-right: solid 6px purple"></tr>
</table>

#### Code

In [11]:
def big_swap(L, left, middle, right, prefix = ''):
    """Swap L[left:middle] with L[middle:right]"""
    print(f'{prefix}* SWAP {L[left:middle]}, {L[middle:right]} WITHIN {L}')
    # Base case: no elements in left part L[left:middle]
    if middle - left < 1: return
    # Base case: no elements in right part L[middle:right]
    if right - middle < 1: return
    # Recursive case.
    if middle <= (left + right) // 2:
        for i in range(left, middle):
            j = i + (middle - left)
            # Swap L[i] and L[j]
            L[i], L[j] = L[j], L[i]
        big_swap(L, middle, middle + (middle - left), right, prefix)
    else:
        for j in reversed(range(middle, right)):
            i = j - (right - middle)
            # Swap L[i] and L[j]
            L[i], L[j] = L[j], L[i]
        big_swap(L, left, middle - (right - middle), middle, prefix)

In [12]:
L = [1, 2, 3, 4]
big_swap(L, 0, 1, 4)
assert L == [2, 3, 4, 1]

* SWAP [1], [2, 3, 4] WITHIN [1, 2, 3, 4]
* SWAP [1], [3, 4] WITHIN [2, 1, 3, 4]
* SWAP [1], [4] WITHIN [2, 3, 1, 4]
* SWAP [1], [] WITHIN [2, 3, 4, 1]


In [13]:
L = [1, 2, 3, 4]
big_swap(L, 0, 2, 4)
assert L == [3, 4, 1, 2]

* SWAP [1, 2], [3, 4] WITHIN [1, 2, 3, 4]
* SWAP [1, 2], [] WITHIN [3, 4, 1, 2]


In [14]:
L = [1, 2, 3, 4]
big_swap(L, 0, 3, 4)
assert L == [4, 1, 2, 3]

* SWAP [1, 2, 3], [4] WITHIN [1, 2, 3, 4]
* SWAP [1, 2], [4] WITHIN [1, 2, 4, 3]
* SWAP [1], [4] WITHIN [1, 4, 2, 3]
* SWAP [1], [] WITHIN [4, 1, 2, 3]


In [15]:
# Lots of testing
for n in range(1, 10):
    for middle in range(n):
        L = list(range(n))
        big_swap(L, 0, middle, n)
        print(L, '==', list(range(middle, n)) + list(range(0, middle)))
        assert L == list(range(middle, n)) + list(range(0, middle))

* SWAP [], [0] WITHIN [0]
[0] == [0]
* SWAP [], [0, 1] WITHIN [0, 1]
[0, 1] == [0, 1]
* SWAP [0], [1] WITHIN [0, 1]
* SWAP [0], [] WITHIN [1, 0]
[1, 0] == [1, 0]
* SWAP [], [0, 1, 2] WITHIN [0, 1, 2]
[0, 1, 2] == [0, 1, 2]
* SWAP [0], [1, 2] WITHIN [0, 1, 2]
* SWAP [0], [2] WITHIN [1, 0, 2]
* SWAP [0], [] WITHIN [1, 2, 0]
[1, 2, 0] == [1, 2, 0]
* SWAP [0, 1], [2] WITHIN [0, 1, 2]
* SWAP [0], [2] WITHIN [0, 2, 1]
* SWAP [0], [] WITHIN [2, 0, 1]
[2, 0, 1] == [2, 0, 1]
* SWAP [], [0, 1, 2, 3] WITHIN [0, 1, 2, 3]
[0, 1, 2, 3] == [0, 1, 2, 3]
* SWAP [0], [1, 2, 3] WITHIN [0, 1, 2, 3]
* SWAP [0], [2, 3] WITHIN [1, 0, 2, 3]
* SWAP [0], [3] WITHIN [1, 2, 0, 3]
* SWAP [0], [] WITHIN [1, 2, 3, 0]
[1, 2, 3, 0] == [1, 2, 3, 0]
* SWAP [0, 1], [2, 3] WITHIN [0, 1, 2, 3]
* SWAP [0, 1], [] WITHIN [2, 3, 0, 1]
[2, 3, 0, 1] == [2, 3, 0, 1]
* SWAP [0, 1, 2], [3] WITHIN [0, 1, 2, 3]
* SWAP [0, 1], [3] WITHIN [0, 1, 3, 2]
* SWAP [0], [3] WITHIN [0, 3, 1, 2]
* SWAP [0], [] WITHIN [3, 0, 1, 2]
[3, 0, 1, 2] =

### All together now!

In [16]:
L = [10, 15, 19, 23, 26, 32, 4, 9, 16, 30, 39, 47]
inplace_merge(L, 0, 6, 12)
assert L == [4, 9, 10, 15, 16, 19, 23, 26, 30, 32, 39, 47]

: MERGE [10, 15, 19, 23, 26, 32] AND [4, 9, 16, 30, 39, 47] WITHIN [10, 15, 19, 23, 26, 32, 4, 9, 16, 30, 39, 47]
:* SWAP [23, 26, 32], [4, 9, 16] WITHIN [10, 15, 19, 23, 26, 32, 4, 9, 16, 30, 39, 47]
:* SWAP [23, 26, 32], [] WITHIN [10, 15, 19, 4, 9, 16, 23, 26, 32, 30, 39, 47]
:: MERGE [10, 15, 19] AND [4, 9, 16] WITHIN [10, 15, 19, 4, 9, 16, 23, 26, 32, 30, 39, 47]
::* SWAP [15, 19], [4, 9] WITHIN [10, 15, 19, 4, 9, 16, 23, 26, 32, 30, 39, 47]
::* SWAP [15, 19], [] WITHIN [10, 4, 9, 15, 19, 16, 23, 26, 32, 30, 39, 47]
::: MERGE [10] AND [4, 9] WITHIN [10, 4, 9, 15, 19, 16, 23, 26, 32, 30, 39, 47]
:::* SWAP [10], [4, 9] WITHIN [10, 4, 9, 15, 19, 16, 23, 26, 32, 30, 39, 47]
:::* SWAP [10], [9] WITHIN [4, 10, 9, 15, 19, 16, 23, 26, 32, 30, 39, 47]
:::* SWAP [10], [] WITHIN [4, 9, 10, 15, 19, 16, 23, 26, 32, 30, 39, 47]
:::: MERGE [] AND [4] WITHIN [4, 9, 10, 15, 19, 16, 23, 26, 32, 30, 39, 47]
:::: MERGE [9, 10] AND [] WITHIN [4, 9, 10, 15, 19, 16, 23, 26, 32, 30, 39, 47]
::: MERGE [15

In [17]:
L = [1, 4, 7, 3]
inplace_merge_sort(L)
assert L == [1, 3, 4, 7]

 SORTING [1, 4, 7, 3]
. SPLIT into [1, 4] and [7, 3]
. SORTING [1, 4]
.. SPLIT into [1] and [4]
.. SORTING [1]
.. SORTING [4]
.. SORTED into [1] and [4]
..: MERGE [1] AND [4] WITHIN [1, 4, 7, 3]
..:* SWAP [], [] WITHIN [1, 4, 7, 3]
..:: MERGE [1] AND [] WITHIN [1, 4, 7, 3]
..:: MERGE [] AND [4] WITHIN [1, 4, 7, 3]
.. MERGED into [1, 4]
. SORTING [7, 3]
.. SPLIT into [7] and [3]
.. SORTING [7]
.. SORTING [3]
.. SORTED into [7] and [3]
..: MERGE [7] AND [3] WITHIN [1, 4, 7, 3]
..:* SWAP [7], [3] WITHIN [1, 4, 7, 3]
..:* SWAP [7], [] WITHIN [1, 4, 3, 7]
..:: MERGE [] AND [3] WITHIN [1, 4, 3, 7]
..:: MERGE [7] AND [] WITHIN [1, 4, 3, 7]
.. MERGED into [3, 7]
. SORTED into [1, 4] and [3, 7]
.: MERGE [1, 4] AND [3, 7] WITHIN [1, 4, 3, 7]
.:* SWAP [4], [3] WITHIN [1, 4, 3, 7]
.:* SWAP [4], [] WITHIN [1, 3, 4, 7]
.:: MERGE [1] AND [3] WITHIN [1, 3, 4, 7]
.::* SWAP [], [] WITHIN [1, 3, 4, 7]
.::: MERGE [1] AND [] WITHIN [1, 3, 4, 7]
.::: MERGE [] AND [3] WITHIN [1, 3, 4, 7]
.:: MERGE [4] AND [7

## Nuts and Bolts Puzzle

A carpenter has a whole collection of nuts and bolts of different sizes in a bag. Each nut is unique and has a corresponding unique bolt of the same size, but the disorganized carpenter has dumped them all into one bag and they are all mixed up.  Given a nut and a bolt, it's easy to try to screw them together and see whether they're perfectly sized for each other, or the bolt is too big (won't screw in), or the bolt is too big (turns loosely).  However, the carpenter's eyes aren't very good, so they can't reliably compare two nuts to each other or two bolts to each other.  How can we best match up the nuts and bolts?

<img src="img/nuts_and_bolts.png">

### Easy Algorithm

Given $n$ nuts and $n$ bolts, the carpenter can pick a nut and try it with every remaining bolt and find the one that fits the nut.  Then, he can put away that nut-bolt pair, and he has a problem of size $n-1$.  This means that he has done $n$ "comparisons" to reduce the problem size by 1.  In the next round, $n-1$ comparisons will shrink the problem size to $n-2$, and so on.  The total number of comparisons required is thus $$n + (n-1) + (n-2) + \cdots + 1 = \frac{n(n+1)}2 > n^2.$$

### Divide and Conquer

Can we do better with divide and conquer?  More concretely, can we split the nuts and bolts up into two sets, each of half the size, so we have two problems of size $n/2$ to work on?  As an added bonus, if the carpenter has a helper, they can work in parallel.

Unfortunately, simply splitting the nuts up into two equal-sized piles $N_1$ and $N_2$ and the bolts into two equal-sized piles $B_1$ and $B_2$ does not work.  If we group nuts and bolts corresponding to $N_1$ and $B_1$ together into a nut-bolt pile, it is quite possible that a nut in $N_1$ may not fit any bolt in $B_1$; the correct bolt might be in $B_2$.

However, what we can do is to pick a bolt, which we call the **pivot bolt**, and use it to determine which nuts are smaller, which one fits exactly, and which nuts are bigger.  We separate the nuts into three piles in this way, with the middle pile being of size 1 and containing the paired nut.  In this process, we have discovered only one nut-bolt pairing, and divided up the nuts. But now we can use the paired nut, which we call the **pivot nut**, to split up the bolts into three piles: the bolts that are bigger than the pivot nut, the pivot bolt, and those that are smaller.  We can group the bolts bigger than the pivot nut with the nuts bigger than the pivot bolt, and group the bolts smaller than the pivot nut with the nuts smaller than the pivot bolt.  Now we have one pile of "big" nuts and "big" bolts, all together, and a pile of "small" nuts and "small" bolts all together.  Depending on the choice of the pivot bolt, there will be a differing number of nuts in the two piles. However, we are guaranteed to have the same number of nuts and bolts in each pile, because the nut corresponding to any bolt in the pile is guaranteed to be in the same pile!

In this strategy, we had to make $n$ comparisons given the pivot bolt to split the nuts into two piles. Then we make $n-1$ comparisons to split the bolts up and add them to the nut piles.  In total, we have made $2n-1$ comparisons so far.  Assuming we chose a pivot nut that was middling in size, we have two problems roughly of size $n/2$, which we can divide again using roughly $n$ comparisons to problems of size $n/4$.   The cool thing is the problem sizes halve at each step, just like merge sort, rather than only shrinking by 1.

For example, suppose $n = 100$.  The original strategy requires 5,050 comparisons. In the new strategy, using 199 comparisons, we get two subproblems each roughly of size 50.  Even if we use the original strategy for each of these subproblems, we will only require 1,225 comparisons for each one, for a total of $199 + 1225 \cdot 2 = 2649$ comparisons.  Of course, we can do a recursive divide-and-conquer, and use even fewer comparisons &mdash; around 700. (In 6.006, you'll see an analysis showing that the comparisons in the new strategy applied recursively grows as $n \log n$ as compared to $n^2$ in the original strategy.)

Interestingly, this puzzle has a deep relationship with perhaps the most widely used sorting algorithm: quicksort.

## Quicksort

Recall in merge sort, we divide the array up into two equal-sized subarrays, and elements in each subarray may be bigger or smaller than elements in the other subarray. Therefore, after sorting the subarrays, we needed a merge operation to find the correct locations for each element in the original array, which was difficult to do in place.

In quicksort, we will spend more time in partitioning the array, but then the merge step will become trivial.  In particular, the algorithm will be naturally in place.

Suppose we have an array with unique elements that we want to sort in ascending order:

<table class="array"><tr><th>a<th>b<th>c<th>d<th>e<th>f<th>g<th>h</tr></table>

We will choose an arbitrary pivot element, say `g`.  Now we will partition the array into three subarrays, where the left subarray has elements less than `g`, the middle subarray has just `g`, and the right subarray has elements greater than `g`.  The left and right subarrays are not sorted.  Thus the array will look like the following:

<table class="array"><tr><th><span style="font-family: roman; font-weight: normal">elements less than</span> g<th>g<th><span style="font-family: roman; font-weight: normal">elements greater than</span> g</tr></table>

We can recursively sort the left subarray without affecting the position of `g`, and similarly for the right subarray.  Once these two subarrays have been sorted, we are done: the already-concatenated results is the desired sorted array.  There is no sophisticated merge step as in merge sort.

### Recursion

Now let's look at the code to implement this algorithm, assuming a function to `partition` the array:

In [18]:
def quicksort(L, left = 0, right = None, depth = 1):
    """Sorts L[left:right] in place, defaulting to all of L."""
    if right is None: right = len(L)
    print('.'*depth, 'QUICKSORT', L[left:right])
    # Base case: less than 2 elements, so already sorted
    if right - left < 2: return
    # Recursive case: at least 2 elements
    # Partition L[left:right] into L[left:middle], L[middle], and L[middle+1:right]
    middle = partition(L, left, right)
    print('.'*depth, f'PARTITIONED INTO {L[left:middle]}, {L[middle]}, {L[middle+1:right]}')
    # Recursively sort L[left:middle]
    quicksort(L, left, middle, depth+1)
    # Recursively sort L[middle+1:right]
    quicksort(L, middle+1, right, depth+1)
    print('.'*depth, 'SORTED INTO', L[left:right])

### Partitioning

Here's one way to implement `partition`.  The idea is to walk through the array from left to right.  If we encounter an element less than the pivot, we continue on.  If we encounter an element greater than the pivot, we need to move it.  We would like to just move it to the end of the array.  To keep everything in-place and not lose any elements, we swap the element with the last element of the array.  In this case, we conceptually shrink the array by 1, and we don't advance our index into the array, because the same index now has a new element to consider (what used to be the last element of the array).  At the end, we need to place the pivot where the array index is.  The simplest way to do this is to choose the pivot to be the original last element of the array; then a single swap gives us what we want.

In [19]:
def partition(L, left, right):
    """Partitions L[left:right] into L[left:middle], L[middle], and L[middle+1:right]
    according to the last element L[right-1] as pivot."""
    # Use the last element as the pivot, and conceptually remove it from L.
    pivot_i = right-1
    pivot = L[pivot_i]
    right -= 1
    # Walk through the array
    i = left
    while i < right:
        print(L[left:i], L[i], L[i+1:right], L[right:pivot_i], L[pivot_i])
        if L[i] <= pivot:
            # If we find an element < pivot, keep it where it is.
            # In this case, we just advance i.
            i += 1
        else:
            # If we find an element > pivot, move it to the end
            # by swapping with the last element of the array.
            L[i], L[right-1] = L[right-1], L[i]
            # In this case, we don't advance i, because we still need
            # to process the swapped element, but we shrink the array.
            right -= 1
    # At the end, i will be the first element > pivot.
    # Put the pivot there by swapping with its current position,
    # which is the end of the array.
    print(L[left:i], L[i] if i < pivot_i else None, L[i+1:pivot_i], L[pivot_i])
    L[i], L[pivot_i] = L[pivot_i], L[i]
    print(L[left:i], L[i], L[i+1:pivot_i+1])
    return i

### Examples

In [20]:
L = [4, 65, 2, -31, -1, 99, 83, 782, 0]
partition(L, 0, len(L))

[] 4 [65, 2, -31, -1, 99, 83, 782] [] 0
[] 782 [65, 2, -31, -1, 99, 83] [4] 0
[] 83 [65, 2, -31, -1, 99] [782, 4] 0
[] 99 [65, 2, -31, -1] [83, 782, 4] 0
[] -1 [65, 2, -31] [99, 83, 782, 4] 0
[-1] 65 [2, -31] [99, 83, 782, 4] 0
[-1] -31 [2] [65, 99, 83, 782, 4] 0
[-1, -31] 2 [] [65, 99, 83, 782, 4] 0
[-1, -31] 2 [65, 99, 83, 782, 4] 0
[-1, -31] 0 [65, 99, 83, 782, 4, 2]


2

In [21]:
L = [1, 4, 7, 3]
quicksort(L)
assert L == [1, 3, 4, 7]

. QUICKSORT [1, 4, 7, 3]
[] 1 [4, 7] [] 3
[1] 4 [7] [] 3
[1] 7 [] [4] 3
[1] 7 [4] 3
[1] 3 [4, 7]
. PARTITIONED INTO [1], 3, [4, 7]
.. QUICKSORT [1]
.. QUICKSORT [4, 7]
[] 4 [] [] 7
[4] None [] 7
[4] 7 []
.. PARTITIONED INTO [4], 7, []
... QUICKSORT [4]
... QUICKSORT []
.. SORTED INTO [4, 7]
. SORTED INTO [1, 3, 4, 7]


In [22]:
L = [23, 3, 45, 7, 6, 11, 14, 12]
quicksort(L)
assert L == [3, 6, 7, 11, 12, 14, 23, 45]

. QUICKSORT [23, 3, 45, 7, 6, 11, 14, 12]
[] 23 [3, 45, 7, 6, 11, 14] [] 12
[] 14 [3, 45, 7, 6, 11] [23] 12
[] 11 [3, 45, 7, 6] [14, 23] 12
[11] 3 [45, 7, 6] [14, 23] 12
[11, 3] 45 [7, 6] [14, 23] 12
[11, 3] 6 [7] [45, 14, 23] 12
[11, 3, 6] 7 [] [45, 14, 23] 12
[11, 3, 6, 7] 45 [14, 23] 12
[11, 3, 6, 7] 12 [14, 23, 45]
. PARTITIONED INTO [11, 3, 6, 7], 12, [14, 23, 45]
.. QUICKSORT [11, 3, 6, 7]
[] 11 [3, 6] [] 7
[] 6 [3] [11] 7
[6] 3 [] [11] 7
[6, 3] 11 [] 7
[6, 3] 7 [11]
.. PARTITIONED INTO [6, 3], 7, [11]
... QUICKSORT [6, 3]
[] 6 [] [] 3
[] 6 [] 3
[] 3 [6]
... PARTITIONED INTO [], 3, [6]
.... QUICKSORT []
.... QUICKSORT [6]
... SORTED INTO [3, 6]
... QUICKSORT [11]
.. SORTED INTO [3, 6, 7, 11]
.. QUICKSORT [14, 23, 45]
[] 14 [23] [] 45
[14] 23 [] [] 45
[14, 23] None [] 45
[14, 23] 45 []
.. PARTITIONED INTO [14, 23], 45, []
... QUICKSORT [14, 23]
[] 14 [] [] 23
[14] None [] 23
[14] 23 []
... PARTITIONED INTO [14], 23, []
.... QUICKSORT [14]
.... QUICKSORT []
... SORTED INTO [14, 23]

### How fast is quicksort?

Quicksort is hard to analyze for algorithmic complexity. This is partly because it is unclear what the sizes of the two subarrays will be. If, in our example, `g` was the largest element in the array, then the left subarray would have $n-1$ elements and the right subarray would have $0$! So in the worst case, quicksort is actually slow (quadratic). Like in the Nuts-and-Bolts puzzle, we want to pick an element with medium size. If we choose the pivot element **randomly** instead of arbitrarily, it will usually be roughly in the middle.  Indeed, then quicksort will perform fast (on the order of $n \log n$ like merge sort). Analyzing the expected running time in this case requires a fancy probabilistic analysis; take 6.046 to see how!